In [3]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm

In [2]:
from fastai.vision import *

In [3]:
learn = load_learner('')

In [4]:
import glob
def make_label_df(addition = 'train'):
    lst = []
    for file in tqdm(glob.glob('data/images_2/GAF/' + addition +'_*.png')):
        idx = int(file.split('_')[-1].split('.')[0])
#         assert type(df.iloc[idx,1]) == np.float32, idx
        
        lst.append({"name":file,
#                     'label':np.float32(df.iloc[idx,1]),
                    'idx':int(idx)})
        
    df= pd.DataFrame().from_dict(lst)
    df.to_csv('data/labels_df_' + addition + '-temp.csv')
    return df

submission = pd.read_csv('data/sample_submission.csv', index_col='seg_id', dtype={"time_to_failure": np.float32})

# Load each test data, create the feature matrix, get numeric prediction
for i, seg_id in enumerate(tqdm(submission.index)):
     make_label_df(seg_id)


100%|██████████| 2624/2624 [1:06:45<00:00,  1.25s/it]


In [6]:
from pyts.image import GramianAngularField, MarkovTransitionField, RecurrencePlot
import matplotlib.pyplot as plt
from skimage.transform import resize

def extract_features(x, method='GAF', image_size=224, graph=False):
    #TODO cmap
    if method == 'GAF':
        transformer = GramianAngularField(image_size=image_size)
    elif method == 'MTF':
        transformer = MarkovTransitionField(image_size=image_size)
    elif method == 'RP':
        transformer = RecurrencePlot(dimension=1, percentage=10)
        
    x = transformer.fit_transform(x[None,:])
    x = resize(x, (1,image_size, image_size))
    return x

def make_pics(x, step_length=1000, method = 'GAF', cmap='rainbow', path = None, additions = '', hard = False):
    if path is None:
        path = "data/images_2/"
    path += method
    if not os.path.isdir(path):
        os.mkdir(path)
    
    # Reshaping and approximate standardization with mean 5 and std 3.
    temp = (x - 5 ) / 3
    
    for i in range(x.shape[0]//(step_length)):        
        file = os.path.join(path, additions + '_' + str(i*step_length) + '.png')
        if not os.path.isfile(file) or hard:
            image_data = extract_features(temp[i:i+step_length,0],method = method)
            plt.imsave(file, image_data[0,:,:], cmap=cmap)

# make_pics(df.to_numpy(),additions = 'train')

In [ ]:
submission = pd.read_csv('data/sample_submission.csv', index_col='seg_id', dtype={"time_to_failure": np.float32})

# Load each test data and create the picture
for i, seg_id in enumerate(tqdm(submission.index)):    
    x = pd.read_csv('data/test/' + seg_id + '.csv').to_numpy()
    make_pics(x, step_length=1000, method = 'GAF', cmap='rainbow', additions = seg_id)
#     if i > 100:
#         break

In [8]:
seg_id='seg_03d680'
x = pd.read_csv('data/test/' + seg_id + '.csv').to_numpy()
make_pics(x, step_length=1000, method = 'GAF', cmap='rainbow', additions = seg_id, hard = True)